# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [2]:
from pyspark.sql import SparkSession

# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName("Spark SQL Quiz") \
    .getOrCreate()
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
# 4) create a view to use with your SQL queries
user_log.createOrReplaceTempView("user_log_table")


# Question 1

Which page did user id ""(empty string) NOT visit?


In [3]:
spark.sql("select page from user_log_table minus select page from user_log_table where userid=''").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [4]:
spark.sql('''
select count(distinct userid) female_users
from user_log_table 
where gender='F'
''').show()

+------------+
|female_users|
+------------+
|         462|
+------------+



# Question 4

How many songs were played from the most played artist?

In [5]:
spark.sql('''
select * from(
select artist,count(*) songs
from user_log_table where page='NextSong'
group by artist order by songs desc 
) X limit 1
''').show()

+--------+-----+
|  artist|songs|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [6]:
is_home = spark.sql('''select userid,page,ts,case when page='Home' then 1 else 0 end home from user_log_table 
where userid !='' and page in('Home','NextSong') 
order by userid, ts desc
''')

is_home.show()

+------+--------+-------------+----+
|userid|    page|           ts|home|
+------+--------+-------------+----+
|    10|NextSong|1513828388284|   0|
|    10|NextSong|1513790894284|   0|
|   100|    Home|1513839673284|   1|
|   100|NextSong|1513776308284|   0|
|   100|    Home|1513776194284|   1|
|   100|NextSong|1513775710284|   0|
|   100|NextSong|1513775556284|   0|
|   100|    Home|1513775431284|   1|
|   100|NextSong|1513750442284|   0|
|   100|NextSong|1513750214284|   0|
|  1000|NextSong|1513720878284|   0|
|  1003|    Home|1513749525284|   1|
|  1003|NextSong|1513749516284|   0|
|  1003|    Home|1513749501284|   1|
|  1005|NextSong|1513782278284|   0|
|  1006|NextSong|1513774019284|   0|
|  1006|NextSong|1513773777284|   0|
|  1006|NextSong|1513773548284|   0|
|  1017|NextSong|1513822643284|   0|
|  1017|    Home|1513822400284|   1|
+------+--------+-------------+----+
only showing top 20 rows



In [7]:
is_home.createOrReplaceTempView("is_home_table")

period = spark.sql('''
select userid,page,ts,home,
sum(home) over(partition by userid order by ts desc) period 
from is_home_table
order by userid,ts desc
''')

period.show()

+------+--------+-------------+----+------+
|userid|    page|           ts|home|period|
+------+--------+-------------+----+------+
|    10|NextSong|1513828388284|   0|     0|
|    10|NextSong|1513790894284|   0|     0|
|   100|    Home|1513839673284|   1|     1|
|   100|NextSong|1513776308284|   0|     1|
|   100|    Home|1513776194284|   1|     2|
|   100|NextSong|1513775710284|   0|     2|
|   100|NextSong|1513775556284|   0|     2|
|   100|    Home|1513775431284|   1|     3|
|   100|NextSong|1513750442284|   0|     3|
|   100|NextSong|1513750214284|   0|     3|
|  1000|NextSong|1513720878284|   0|     0|
|  1003|    Home|1513749525284|   1|     1|
|  1003|NextSong|1513749516284|   0|     1|
|  1003|    Home|1513749501284|   1|     2|
|  1005|NextSong|1513782278284|   0|     0|
|  1006|NextSong|1513774019284|   0|     0|
|  1006|NextSong|1513773777284|   0|     0|
|  1006|NextSong|1513773548284|   0|     0|
|  1017|NextSong|1513822643284|   0|     0|
|  1017|    Home|1513822400284| 

In [9]:
period.createOrReplaceTempView("period_table")

spark.sql('''
select avg(songs) from(
select userid,period,count(*) songs
from period_table where page = 'NextSong'
group by userid,period
)'''
).show()

+-----------------+
|       avg(songs)|
+-----------------+
|6.898347107438017|
+-----------------+

